Alright here's whats going down

*1.    Load data in*

*2.    Take a peek to make sure that the resized data maintained its order
*
*3.    Stratify split the data into 5 different train val sets*
      
4.    Using the functional keras api let the meta data and images work together to predict melanoma
      - efn4 & dense network
      
5.    Train the model on original data first to get an idea of what hyperparameters to set
      - adjust network as necessary 

6.    Make predictions on the test set

7.    Submit that stuff

In [ ]:
!pip install -U efficientnet

In [ ]:
# Importing modules
import numpy as np 
import pandas as pd 
import os
from keras.preprocessing.image import load_img, img_to_array
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.metrics import AUC
from keras.layers import *
import efficientnet.keras as efn
import matplotlib.pyplot as plt
import keras

In [ ]:
# Loading in the data
img_dir = "../input/siimisic-melanoma-resized-images"
X_img_train = np.load(os.path.join(img_dir, "x_train_224.npy"))
y_train = pd.read_csv("../input/siim-isic-melanoma-classification/train.csv")['target'].values
X_meta_train = np.load("../input/melanoma-processed-data/preprocessed_meta.npz", allow_pickle=True)

In [ ]:
# Loading in preprocessed metadata
train_meta, test_meta, _ = [X_meta_train[f] for f in X_meta_train.files]
del X_meta_train, _

In [ ]:
# Stratify splitting
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=5, test_size=.1)

# Getting indices to split the train and val sets. Splitting later to save memory
train_ind, cv_ind = [], []
for train_index, cv_index in sss.split(train_meta, y_train):
    train_ind.append(train_index)
    cv_ind.append(cv_index)

In [ ]:
# Loading in efn4 network
efn4 = efn.EfficientNetB4(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

In [ ]:
# Trying to build model
img_input = keras.Input(shape=(224,224,3), name="img_input")
meta_input = keras.Input(shape=(9,), name='meta_input')

# Creating cnn part of network
cnn_features = efn4(img_input)
cnn_features = GlobalAveragePooling2D()(cnn_features)

# Creating dense metadata learning part
meta_features = Dense(200, activation='relu')(meta_input)
meta_features = Dropout(.5)(meta_features)
meta_features = Dense(100, activation='relu')(meta_features)
meta_features = Dropout(.5)(meta_features)

# Merging features
x = concatenate([cnn_features, meta_features])

# Putting the top on the model
conc_features = Dense(1024, activation='relu')(x)
conc_features = Dropout(.5)(conc_features)
output = Dense(1, activation='sigmoid')(conc_features)

# The model
model = keras.Model(inputs=[img_input, meta_input], outputs=output)

In [ ]:
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# Compiling the model
model.compile(optimizer=Adam(.00001), loss='binary_crossentropy', metrics=[AUC(name="auc")])

# Callback saving best weights 
mcp_save = ModelCheckpoint('model_weights0.hdf5', save_best_only=True, monitor='val_auc', mode='max')

In [ ]:
# Lets try training this model
model.fit({"meta_input": train_meta, "img_input": X_img_train}, y_train, epochs=4, validation_split=.1, callbacks=[mcp_save],
         verbose=2)